In [24]:
!pip install vk_api

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.6/48.6 kB 721.4 kB/s eta 0:00:000:00:01


In [104]:
#import needed moduls 
from bs4 import BeautifulSoup
import pandas as pd
import requests
import json 
import time
import vk_api
import tqdm
import matplotlib.pyplot as plt

In [95]:
vk_session = vk_api.VkApi(token=token)
vk = vk_session.get_api()

<h2>API VK

In [93]:
api_version = '5.199'      # lastes api version
client_id = '####' # client_id of my app

scope = 'friends,wall,status,wall'                  # scope parameters
print('\Your API VK generation link: \n')
print(f'https://oauth.vk.com/authorize?client_id={client_id}&display=page&scope={scope}&redirect_uri=https://oauth.vk.com/blank.html&response_type=token&v={api_version}&state=123456')


\Your API VK generation link: 

https://oauth.vk.com/authorize?client_id=51923427&display=page&scope=friends,wall,status,wall&redirect_uri=https://oauth.vk.com/blank.html&response_type=token&v=5.199&state=123456


In [2]:
token = '####'

<h2> Link parsing function</h2>

Firstly, parse links of Moscow groups

In [36]:
search = pd.DataFrame()
a = 0
while a < 300:
    tm_search = pd.json_normalize(vk.groups.search(q = 'Москв', offset = a, count = 50)['items'])
    a = a + 50
    search = pd.concat([tm_search.iloc[:, :2], search])

In [43]:
search

,id,name
0,18164396,"Эстетический массаж, коррекция фигуры Москва СПб"
1,222784721,МОСКВА объявления | РАБОТА
2,309403,Знакомства Москва
3,220370783,РАБОТА МОСКВА
4,33118325,Парки Москвы
...,...,...
45,49673019,Москва FM
46,42334351,ГУ МЧС России по г. Москве
47,210924525,Protasova Mariya | Женская одежда | Москва
48,41438670,Работа в Москве


<h2>Commentaries parsing function</h2>

Secondly, parse a commentaries from posts in *links_coo*

In [72]:
def get_id(link):
    '''
    This functions transform vk link to id
    
    Arguments:
        > link - str - a full vk link
    Returns:
        > id - str - a group id
    '''
    name = link.split('/')[-1]
    id = "-" + str(vk.groups.getById(group_id = name, fields = 'id')[0]['id']) #use "-" only for groups
    return id
def parse_comments(owner_id = '-184752831', post_id = '2036'):
    '''
    This fuction using VK API to parse a text of comments in community from link and returns a pandas.DataFrame

    Arguments:
        > owner_id - str -  an id of group
        > post_id - str - a post id
    Returns:
        >  comments - pandas.DataFrame - with users id and comment text:
            : user_id - str - id
            : comment - str - text comment 
    '''
    comments_counter = vk.wall.getComments(owner_id = owner_id, post_id = post_id)['count']
    comments = pd.DataFrame()
    temp_df = pd.DataFrame()
    comments_data = vk.wall.getComments(owner_id=owner_id, post_id=post_id)['items']
    if comments_counter > 0:
        for i in tqdm.tqdm(range(min(comments_counter, len(comments_data)))):
            temp_df = pd.json_normalize(vk.wall.getComments(owner_id = owner_id, post_id = post_id)['items'][i])
            comments = pd.concat([temp_df, comments])
    return comments
    
def get_posts(link, count = 10):
    '''
    This function get group vk link and return all comments from first 10 (by default) posts 

    Arguments:
        > link - str - full link to group
        > count - int - count of first posts (10, by default)
    '''
    owner_id = "-" + str(id) #get_id(link) -- if we have an full link
    posts = vk.wall.get(owner_id=owner_id, count=count)['items']
    post_ids = [i['id'] for i in posts]
    answer = pd.DataFrame()
    for i in tqdm.tqdm(post_ids):
        tm_answer = parse_comments(owner_id = owner_id, post_id = i)
        answer = pd.concat([tm_answer,answer])
        time.sleep(0.5)
    return answer


<h3>Let's try at links</h3>

In [74]:
full_df = pd.DataFrame()

for id in tqdm.tqdm(search['id']):
    try:
        temp_df = get_posts(str(id), 20)
        full_df = pd.concat([temp_df, full_df])
    except:
        print('problem:', id)
full_df = full_df[['from_id', 'text']]

  0%|                                                    | 0/20 [00:00<?, ?it/s]

  0%|                                                     | 0/4 [00:00<?, ?it/s]

 25%|███████████▎                                 | 1/4 [00:00<00:01,  2.48it/s]

 50%|██████████████████████▌                      | 2/4 [00:00<00:00,  2.46it/s]

 75%|█████████████████████████████████▊           | 3/4 [00:01<00:00,  2.43it/s]

100%|█████████████████████████████████████████████| 4/4 [00:01<00:00,  2.44it/s]

  5%|██▏                                         | 1/20 [00:02<00:56,  2.99s/it]

  0%|                                                     | 0/5 [00:00<?, ?it/s]

 20%|█████████                                    | 1/5 [00:00<00:01,  2.37it/s]

 40%|██████████████████                           | 2/5 [00:00<00:01,  2.43it/s]

 60%|███████████████████████████                  | 3/5 [00:01<00:00,  2.43it/s]

 80%|████████████████████████████████████         | 4/5 [00:01<00:00,  2.43it/s]

100%|███████████

problem: 108774662



 20%|████████▊                                   | 4/20 [00:04<00:16,  1.00s/it]

  0%|                                                     | 0/1 [00:00<?, ?it/s]

100%|█████████████████████████████████████████████| 1/1 [00:00<00:00,  2.40it/s]

 50%|█████████████████████▌                     | 10/20 [00:10<00:09,  1.02it/s]

  0%|                                                     | 0/1 [00:00<?, ?it/s]

100%|█████████████████████████████████████████████| 1/1 [00:00<00:00,  2.51it/s]

 65%|███████████████████████████▉               | 13/20 [00:13<00:07,  1.03s/it]

  0%|                                                     | 0/1 [00:00<?, ?it/s]

100%|█████████████████████████████████████████████| 1/1 [00:00<00:00,  2.39it/s]

 70%|██████████████████████████████             | 14/20 [00:15<00:06,  1.14s/it]

  0%|                                                     | 0/4 [00:00<?, ?it/s]

 25%|███████████▎                                 | 1/4 [00:00<00:01,  2.46it/s]

 50%|██████████

problem: 222784685


 25%|██████████▏                             | 76/300 [52:10<2:14:47, 36.11s/it]

...

problem: 210924525


100%|████████████████████████████████████████▊| 299/300 [53:50<00:00,  2.16it/s]

problem: 41438670


100%|█████████████████████████████████████████| 300/300 [53:50<00:00, 10.77s/it]

problem: 211206852


In [75]:
full_df

,from_id,text
0,669275356,Я сам из Перми а хочу очень в Москву
0,-213613520,
0,3591505,"Да,Москоу-сити с любого ракурса интересно смот..."
0,-213613520,"Часто работаем в Москва Сити, отличный район 👍🏻"
0,105897545,89!!!
...,...,...
0,215677196,
0,44284286,✅ ЧИТАЙТЕ САМЫЕ ЗАБАВНЫЕ И ИНТЕРЕСНЫЕ ОТЗЫВЫ:\...
0,44284286,✅ ЧИТАЙТЕ САМЫЕ ЗАБАВНЫЕ И ИНТЕРЕСНЫЕ ОТЗЫВЫ:\...
0,78295055,Милота


In [113]:
full_df.to_csv('ids_and_comments_moscow.csv')

<h2>Oops, we have some problem links</h2>
I think our problem based on api restrictions, so let's try to collect this ids and then start our function again

In [76]:
id = input()

 problem: 222784685   25%|██████████▏                             | 76/300 [52:10<2:14:47, 36.11s/it]  problem: 196613804   26%|██████████▎                             | 77/300 [52:11<1:34:25, 25.40s/it]  problem: 222784636   26%|██████████▍                             | 78/300 [52:11<1:06:17, 17.92s/it]  problem: 176335500   26%|███████████                               | 79/300 [52:11<46:39, 12.67s/it]  problem: 35834229   27%|███████████▏                              | 80/300 [52:12<32:58,  8.99s/it]  problem: 202440785   27%|███████████▎                              | 81/300 [52:12<23:27,  6.43s/it]  problem: 222784650   27%|███████████▍                              | 82/300 [52:13<16:48,  4.62s/it]  problem: 222784709   28%|███████████▌                              | 83/300 [52:13<12:09,  3.36s/it]  problem: 9166679   28%|███████████▊                              | 84/300 [52:14<08:56,  2.48s/it]  problem: 74493680   28%|███████████▉                              | 85/300 [52:14<06

In [87]:
#bad_id = [i.split()[0] for i in id.split('problem: ')]
bad_id = []
for i in id.split('problem: '):
    try:
        bad_id.append(i.split()[0])
    except:
        print(i)

In [88]:
bad_id 

['222784685',
 '196613804',
 '222784636',
 '176335500',
 '35834229',
 '202440785',
 '222784650',
 '222784709',
 '9166679',
 '74493680',
 '222784682',
 '49253163',
 '207472059',
 '162260077',
 '123138388',
 '76231763',
 '187978780',
 '52053186',
 '187978511',
 '218265750',
 '13194512',
 '171169255',
 '102796435',
 '196614377',
 '69936917',
 '218318195',
 '66768541',
 '222784672',
 '37637089',
 '115718282',
 '164592450',
 '174164907',
 '3291129',
 '50204926',
 '218265775',
 '84724704',
 '220045598',
 '187882693',
 '218265846',
 '90870698',
 '189271885',
 '122965436',
 '222784706',
 '116817225',
 '222784673',
 '221528381',
 '169705869',
 '87724110',
 '196613689',
 '222784638',
 '91728303',
 '218265883',
 '185420720',
 '166856580',
 '219548972',
 '16565988',
 '35525659',
 '71558031',
 '222784620',
 '185729570',
 '222784604',
 '191699171',
 '196613768',
 '57800741',
 '183953086',
 '187978802',
 '218265737',
 '58053809',
 '139111933',
 '222784644',
 '196613751',
 '63346315',
 '222784625',
 '

*second iteration of our functin:*

In [98]:
for id in tqdm.tqdm(bad_id):
    try:
        temp_df = get_posts(str(id), 20)
        full_df = pd.concat([temp_df, full_df])
    except Exception as e:
        print('problem:', id, e)
        bad_links_2.append(bad_id)
full_df = full_df[['from_id', 'text']]

  0%|                                                    | 0/20 [00:00<?, ?it/s]

  0%|                                                    | 0/10 [00:00<?, ?it/s]

 10%|████▍                                       | 1/10 [00:00<00:05,  1.65it/s]

 20%|████████▊                                   | 2/10 [00:01<00:04,  1.86it/s]

 30%|█████████████▏                              | 3/10 [00:01<00:03,  1.97it/s]

 40%|█████████████████▌                          | 4/10 [00:02<00:03,  1.95it/s]

 50%|██████████████████████                      | 5/10 [00:02<00:02,  1.93it/s]

 60%|██████████████████████████▍                 | 6/10 [00:03<00:02,  1.83it/s]

 70%|██████████████████████████████▊             | 7/10 [00:03<00:01,  1.94it/s]

 80%|███████████████████████████████████▏        | 8/10 [00:04<00:01,  1.93it/s]

 90%|███████████████████████████████████████▌    | 9/10 [00:04<00:00,  1.93it/s]

100%|███████████████████████████████████████████| 10/10 [00:05<00:00,  1.91it/s]

  5%|██▏        

problem: 63346315 [15] Access denied: this wall available only for community members



  0%|                                                    | 0/20 [00:00<?, ?it/s]

  0%|                                                    | 0/10 [00:00<?, ?it/s]

 10%|████▍                                       | 1/10 [00:00<00:04,  1.90it/s]

 20%|████████▊                                   | 2/10 [00:01<00:03,  2.02it/s]

 30%|█████████████▏                              | 3/10 [00:01<00:03,  1.95it/s]

 40%|█████████████████▌                          | 4/10 [00:02<00:03,  1.94it/s]

 50%|██████████████████████                      | 5/10 [00:02<00:02,  1.93it/s]

 60%|██████████████████████████▍                 | 6/10 [00:03<00:02,  1.92it/s]

 70%|██████████████████████████████▊             | 7/10 [00:03<00:01,  1.91it/s]

 80%|███████████████████████████████████▏        | 8/10 [00:04<00:01,  1.91it/s]

 90%|███████████████████████████████████████▌    | 9/10 [00:04<00:00,  1.71it/s]

100%|███████████████████████████████████████████| 10/10 [00:05<00:00,  1.84it/s]

  5%|██▏       

problem: 255 [15] Access denied: this wall available only for community members



  0%|                                                    | 0/20 [00:00<?, ?it/s]

  0%|                                                    | 0/10 [00:00<?, ?it/s]

 10%|████▍                                       | 1/10 [00:00<00:04,  1.90it/s]

 20%|████████▊                                   | 2/10 [00:00<00:03,  2.04it/s]

 30%|█████████████▏                              | 3/10 [00:01<00:03,  1.92it/s]

 40%|█████████████████▌                          | 4/10 [00:02<00:03,  1.95it/s]

 50%|██████████████████████                      | 5/10 [00:02<00:02,  1.95it/s]

 60%|██████████████████████████▍                 | 6/10 [00:03<00:02,  1.93it/s]

 70%|██████████████████████████████▊             | 7/10 [00:03<00:01,  1.91it/s]

 80%|███████████████████████████████████▏        | 8/10 [00:04<00:01,  1.92it/s]

 90%|███████████████████████████████████████▌    | 9/10 [00:04<00:00,  1.91it/s]

100%|███████████████████████████████████████████| 10/10 [00:05<00:00,  1.93it/s]

 10%|████▍     

...


 10%|████▍                                       | 2/20 [00:02<00:22,  1.24s/it]

  0%|                                                     | 0/1 [00:00<?, ?it/s]

100%|█████████████████████████████████████████████| 1/1 [00:00<00:00,  1.92it/s]

 15%|██████▌                                     | 3/20 [00:04<00:25,  1.48s/it]

  0%|                                                     | 0/1 [00:00<?, ?it/s]

100%|█████████████████████████████████████████████| 1/1 [00:00<00:00,  2.44it/s]

 20%|████████▊                                   | 4/20 [00:05<00:23,  1.50s/it]

  0%|                                                     | 0/1 [00:00<?, ?it/s]

100%|█████████████████████████████████████████████| 1/1 [00:00<00:00,  1.80it/s]

 25%|███████████                                 | 5/20 [00:08<00:27,  1.85s/it]

  0%|                                                     | 0/2 [00:00<?, ?it/s]

 50%|██████████████████████▌                      | 1/2 [00:00<00:00,  2.00it/s]

100%|██████████

problem: 138520920 [15] Access denied: this wall available only for community members



 15%|██████▌                                     | 3/20 [00:03<00:21,  1.26s/it]

  0%|                                                     | 0/1 [00:00<?, ?it/s]

100%|█████████████████████████████████████████████| 1/1 [00:00<00:00,  2.22it/s]

  0%|                                                    | 0/20 [00:00<?, ?it/s]

  0%|                                                    | 0/10 [00:00<?, ?it/s]

 10%|████▍                                       | 1/10 [00:00<00:04,  2.06it/s]

 20%|████████▊                                   | 2/10 [00:01<00:04,  1.89it/s]

 30%|█████████████▏                              | 3/10 [00:01<00:03,  1.92it/s]

 40%|█████████████████▌                          | 4/10 [00:02<00:03,  1.96it/s]

 50%|██████████████████████                      | 5/10 [00:02<00:02,  1.90it/s]

 60%|██████████████████████████▍                 | 6/10 [00:03<00:02,  1.94it/s]

 70%|██████████████████████████████▊             | 7/10 [00:03<00:01,  1.84it/s]

 80%|██████████

problem: 2781 [15] Access denied: this wall available only for community members



  0%|                                                    | 0/20 [00:00<?, ?it/s]

  0%|                                                     | 0/1 [00:00<?, ?it/s]

100%|█████████████████████████████████████████████| 1/1 [00:00<00:00,  1.90it/s]

  5%|██▏                                         | 1/20 [00:02<00:42,  2.23s/it]

  0%|                                                     | 0/3 [00:00<?, ?it/s]

 33%|███████████████                              | 1/3 [00:00<00:00,  2.35it/s]

 67%|██████████████████████████████               | 2/3 [00:00<00:00,  2.29it/s]

100%|█████████████████████████████████████████████| 3/3 [00:01<00:00,  2.16it/s]

 10%|████▍                                       | 2/20 [00:04<00:43,  2.40s/it]

  0%|                                                     | 0/3 [00:00<?, ?it/s]

 33%|███████████████                              | 1/3 [00:00<00:01,  1.67it/s]

 67%|██████████████████████████████               | 2/3 [00:01<00:00,  2.03it/s]

100%|██████████

In [111]:
full_df = full_df[full_df['text'] != '']

In [112]:
full_df #our full dataset

,from_id,text
0,-45186843,"""Метрополитен мог бы удешевить стоимость, пото..."
0,183933233,"Дорого 130 р., за 100 мл.. Нигде не написано, ..."
0,183933233,Жаль детей.
0,101705512,Какая дурь. Кофе то причём?
0,484178458,а вейпы - реклпама на каждой остановке и продажа
...,...,...
0,80538473,"Ну, и ходите на массаж!"
0,169486537,"А она ему : ""Сижу рядом с лысым коротыжкой с б..."
0,44284286,✅ ЧИТАЙТЕ САМЫЕ ЗАБАВНЫЕ И ИНТЕРЕСНЫЕ ОТЗЫВЫ:\...
0,44284286,✅ ЧИТАЙТЕ САМЫЕ ЗАБАВНЫЕ И ИНТЕРЕСНЫЕ ОТЗЫВЫ:\...


<h2>Some dataset description</h2>

In [103]:
len(full_df['from_id']) - len(full_df['from_id'].unique() #not-unique ids 

5661